In [0]:
%sql

USE my_league_db;

-- 1. Insert sample addresses
INSERT INTO Address VALUES
  (1, '123 Main St', 'Provo', 'UT', '84601'),
  (2, '456 Center St', 'Orem', 'UT', '84057'),
  (3, '789 State St', 'Lehi', 'UT', '84043');

-- 2. Insert sample venues, each referencing an address
INSERT INTO Venue VALUES
  (1, 'Stadium A', 1, 1),
  (2, 'Stadium B', 2, 1),
  (3, 'Stadium C', 3, 1);

-- 3. Insert 3 divisions
INSERT INTO Division VALUES
  (1, 'Division A'),
  (2, 'Division B'),
  (3, 'Division C');

-- 4. Insert 24 teams (8 per division)
INSERT INTO Team VALUES
  (1, 'Team 1', 1), (2, 'Team 2', 1), (3, 'Team 3', 1), (4, 'Team 4', 1),
  (5, 'Team 5', 1), (6, 'Team 6', 1), (7, 'Team 7', 1), (8, 'Team 8', 1),
  (9, 'Team 9', 2), (10, 'Team 10', 2), (11, 'Team 11', 2), (12, 'Team 12', 2),
  (13, 'Team 13', 2), (14, 'Team 14', 2), (15, 'Team 15', 2), (16, 'Team 16', 2),
  (17, 'Team 17', 3), (18, 'Team 18', 3), (19, 'Team 19', 3), (20, 'Team 20', 3),
  (21, 'Team 21', 3), (22, 'Team 22', 3), (23, 'Team 23', 3), (24, 'Team 24', 3);

-- 5. Insert 405 people (360 players, 45 referees)
-- Players: byuId 1000000-1000359
INSERT INTO Person
SELECT
  1000000 + idx AS byuId,
  CONCAT('Player', 1000000 + idx) AS firstName,
  CONCAT('Last', 1000000 + idx) AS lastName,
  CONCAT('player', 1000000 + idx, '@example.com') AS email,
  CONCAT('555-01', LPAD(CAST(idx AS STRING), 4, '0')) AS phone
FROM (
  SELECT posexplode(sequence(0, 359)) AS (idx, val)
);

-- Referees: byuId 1000360-1000404
INSERT INTO Person
SELECT
  1000360 + idx AS byuId,
  CONCAT('Ref', 1000360 + idx) AS firstName,
  CONCAT('Last', 1000360 + idx) AS lastName,
  CONCAT('ref', 1000360 + idx, '@example.com') AS email,
  CONCAT('555-02', LPAD(CAST(idx AS STRING), 4, '0')) AS phone
FROM (
  SELECT posexplode(sequence(0, 44)) AS (idx, val)
);

-- 6. Insert 360 players (15 per team)
INSERT INTO Player
SELECT
  1 + idx AS playerId,
  1000000 + idx AS byuId,
  1 + CAST(idx / 15 AS INT) AS teamId
FROM (
  SELECT posexplode(sequence(0, 359)) AS (idx, val)
);

-- 7. Insert 45 referees (15 per division)
INSERT INTO Referee
SELECT
  1 + idx AS refereeId,
  1000360 + idx AS byuId
FROM (
  SELECT posexplode(sequence(0, 44)) AS (idx, val)
);

-- Division 1: Teams 1-8, Referees 1-15
INSERT INTO Game
SELECT
  row_number() OVER (ORDER BY t1.teamId, t2.teamId) AS gameId,
  1 AS divisionId,
  t1.teamId AS homeTeamId,
  t2.teamId AS awayTeamId,
  date_add(timestamp('2024-07-01 18:00:00'), row_number() OVER (ORDER BY t1.teamId, t2.teamId)) AS dateTime,
  1 AS venueId,
  CAST(floor(rand() * 5) + 1 AS INT) AS homeTeamRating,
  CAST(floor(rand() * 5) + 1 AS INT) AS awayTeamRating,
  1 + (row_number() OVER (ORDER BY t1.teamId, t2.teamId) % 15) AS referee1Id,
  1 + ((row_number() OVER (ORDER BY t1.teamId, t2.teamId) + 1) % 15) AS referee2Id
FROM Team t1
JOIN Team t2
  ON t1.divisionId = 1 AND t2.divisionId = 1 AND t1.teamId < t2.teamId;

  -- Division 2: Teams 9-16, Referees 16-30, Venue 2
INSERT INTO Game
SELECT
  28 + row_number() OVER (ORDER BY t1.teamId, t2.teamId) AS gameId,
  2 AS divisionId,
  t1.teamId AS homeTeamId,
  t2.teamId AS awayTeamId,
  date_add(timestamp('2024-07-29 18:00:00'), row_number() OVER (ORDER BY t1.teamId, t2.teamId)) AS dateTime,
  2 AS venueId,
  CAST(floor(rand() * 5) + 1 AS INT) AS homeTeamRating,
  CAST(floor(rand() * 5) + 1 AS INT) AS awayTeamRating,
  16 + (row_number() OVER (ORDER BY t1.teamId, t2.teamId) % 15) AS referee1Id,
  16 + ((row_number() OVER (ORDER BY t1.teamId, t2.teamId) + 1) % 15) AS referee2Id
FROM Team t1
JOIN Team t2
  ON t1.divisionId = 2 AND t2.divisionId = 2 AND t1.teamId < t2.teamId;

-- Division 3: Teams 17-24, Referees 31-45, Venue 3
INSERT INTO Game
SELECT
  56 + row_number() OVER (ORDER BY t1.teamId, t2.teamId) AS gameId,
  3 AS divisionId,
  t1.teamId AS homeTeamId,
  t2.teamId AS awayTeamId,
  date_add(timestamp('2024-08-26 18:00:00'), row_number() OVER (ORDER BY t1.teamId, t2.teamId)) AS dateTime,
  3 AS venueId,
  CAST(floor(rand() * 5) + 1 AS INT) AS homeTeamRating,
  CAST(floor(rand() * 5) + 1 AS INT) AS awayTeamRating,
  31 + (row_number() OVER (ORDER BY t1.teamId, t2.teamId) % 15) AS referee1Id,
  31 + ((row_number() OVER (ORDER BY t1.teamId, t2.teamId) + 1) % 15) AS referee2Id
FROM Team t1
JOIN Team t2
  ON t1.divisionId = 3 AND t2.divisionId = 3 AND t1.teamId < t2.teamId;

num_affected_rows,num_inserted_rows
28,28
